# Liver Model Construction: Huge Core Model

In [1]:
import os
import re
import warnings
from cobra.io import *
from cobra.io.json import *
import escher
import mass
import numpy as np
import pandas as pd
import equilibrator_api
from equilibrator_api import ComponentContribution, Q_
import sympy as sym
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass.util import qcqa, qcqa_model
from cobra import DictList
from mass import (
    MassConfiguration, MassMetabolite, MassModel,
    MassReaction, Simulation, UnitDefinition)
from mass.io.json import save_json_model as save_json_mass_model
from mass.visualization import plot_comparison, plot_time_profile, plot_ensemble_phase_portrait, plot_ensemble_time_profile
mass_config = MassConfiguration()
mass_config.irreversible_Keq = float("inf")
print("MASSpy version: {0}".format(mass.__version__))
from six import iteritems
import matplotlib.pyplot as plt
from mass.thermo import (
    ConcSolver, sample_concentrations,
    update_model_with_concentration_solution)
# adding Folder_2 to the system path
sys.path.insert(0, "c:\\Users\\sicil\\LiverModel")
from cobra_dict import *
import cobra_dict as c_d
import csv
import altair as alt
from minspan.minspan import minspan,nnz
import sys
# from minspan import minspan,nnz

MASSpy version: 0.1.6


In [2]:
mass.__version__

'0.1.6'

In [3]:
maps_dir = os.path.abspath("maps")
data_dir = os.path.abspath("data")
dir = os.path.abspath("")
model_dir = os.path.abspath("models")
minspan_dir= os.path.abspath("minspans_csv")


### Getting reactions and metabolite data from RECON 3D

In [4]:
# making dataframe of metabolites

csv_met = os.path.join(data_dir,"big_core_met_df.csv")
met_csv_df = pd.read_csv(csv_met,index_col="id")
metabolite_info=met_csv_df.to_dict(orient='index')


### Loading cobra model

In [5]:
# Initiate empty model
trial= cobra.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-02


In [6]:
#  Add all the remaining metabolites involved in the pathway
for met_id, info in metabolite_info.items():
    met = cobra.Metabolite(met_id, name=info['name'], formula=info['formula'], 
                           charge=info['charge'], compartment=info['compartment'])
    trial.add_metabolites(met)

In [7]:
# Loading reaction data as JSON file to maintain data types as dictionaries 
rxn_json = os.path.join(data_dir,"big_core_reaction_df.json")
with open(rxn_json) as json_file:
    rxn_data = json.load(json_file)

In [8]:
for reaction, info in rxn_data.items():
    reaction_obj = cobra.Reaction(reaction)
    reaction_obj.id=reaction
    reaction= reaction_obj.id
    reaction_obj.lower_bound = info['lower_bound']
    reaction_obj.upper_bound = info['upper_bound']
    reaction_obj.name = info['name']
    trial.add_reaction(reaction_obj)
    temp=info['metabolites']
    reaction_obj.add_metabolites(temp)
    # print(reaction)

In [9]:
#Mass balance check
for r in trial.reactions:
    print(r.id,  r.check_mass_balance())

CSm {'charge': -1.0}
ACONTm {}
ICDHxm {}
AKGDm {}
SUCOASm {'charge': 2.0}
SUCD1m {}
FUMm {}
MDHm {'charge': -1.0}
NADH2_u10mi {'charge': 1.0}
FADH2ETC {}
CYOR_u10mi {'charge': -2.0}
CYOOm2i {'charge': 4.0}
ATPS4mi {'charge': -1.0}
Htmi {}
HEX1 {'charge': -1.0}
PGI {}
PFK {'charge': -1.0}
FBA {}
TPI {}
GAPD {'charge': -3.0}
PGK {}
PGM {}
ENO {}
PYK {'charge': 1.0}
ADK1 {}
PDHm {}
LDH_L {'charge': -1.0}
G6PDH2r {'charge': -1.0}
PGL {'charge': -1.0}
GND {}
RPI {}
RPE {}
TKT1 {}
TKT2 {}
TALA {}
PCm {'charge': 1.0}
PEPCKm {}
FBP {'charge': 2.0}
G6PPer {'charge': 2.0}
G6Pter {}
GLCter {}
MALtm {}
MDH {'charge': -1.0}
PEPCK_re {}
PGMT {}
GALUi {'charge': 1.0}
GLGNS1 {'charge': -3.0}
GLBRAN {}
GLPASE1 {'charge': -6.0}
GLDBRAN {}
GLPASE2 {}
GGNG {'charge': -8.0}
ACACT1m {}
HMGCOASm {'charge': 1.0}
HMGLm {}
ADCim {'charge': 1.0}
BDHm {'charge': -1.0}
OCOAT1m {}
NDPK1 {}
NDPK1m {}
NDPK2 {}
HCO3Em {'charge': -1.0}
ATPtm {}
AKGMALtm {}
ASPTA {}
ASPTAm {}
CITtam {}
CITL {}
ME2 {}
ACS {}
ARGN {}
OCBT

In [10]:
trial.metabolites.h_c.charge= 1
trial.metabolites.h_m.charge= 1
trial.metabolites.h_i.charge= 1
trial.metabolites.pi_c.charge= -2
trial.metabolites.pi_m.charge= -2

In [11]:
for r in trial.reactions:
    print(r.id,  r.check_mass_balance())

CSm {}
ACONTm {}
ICDHxm {}
AKGDm {}
SUCOASm {}
SUCD1m {}
FUMm {}
MDHm {}
NADH2_u10mi {}
FADH2ETC {}
CYOR_u10mi {}
CYOOm2i {}
ATPS4mi {}
Htmi {}
HEX1 {}
PGI {}
PFK {}
FBA {}
TPI {}
GAPD {}
PGK {}
PGM {}
ENO {}
PYK {}
ADK1 {}
PDHm {}
LDH_L {}
G6PDH2r {}
PGL {}
GND {}
RPI {}
RPE {}
TKT1 {}
TKT2 {}
TALA {}
PCm {}
PEPCKm {}
FBP {}
G6PPer {'charge': 2.0}
G6Pter {}
GLCter {}
MALtm {}
MDH {}
PEPCK_re {}
PGMT {}
GALUi {}
GLGNS1 {}
GLBRAN {}
GLPASE1 {}
GLDBRAN {}
GLPASE2 {}
GGNG {}
ACACT1m {}
HMGCOASm {}
HMGLm {}
ADCim {}
BDHm {}
OCOAT1m {}
NDPK1 {}
NDPK1m {}
NDPK2 {}
HCO3Em {}
ATPtm {}
AKGMALtm {}
ASPTA {}
ASPTAm {}
CITtam {}
CITL {}
ME2 {}
ACS {}
ARGN {}
OCBTm {}
ARGSS {}
ARGSL {}
CBPSam {}
NH4tm {}
GDHm {}
GLNS {}
GLUN {}
ALATA_L {}
CYSO {}
3SALATAi {}
3SPYRSP {}
PGCD {}
PSERT {}
PSP_L {}
GHMT2r {}
ASNS1 {}
GLYtm {}
MLTHFtm {}
THFtm {}
KHK {}
FBA2 {}
TRIOK {}
ALCD19y {}
GLYK {}
2AMACHYD {}
SERHL {}
ALCD2x {}
ACALDtm {}
ALDD2xm {}
ACSm {}
Htm {}
O2tm {}
H2Otr {}
H2Otm {}
PItm {}
PItr {'charge'

In [12]:
trial.objective = 'ATPS4mi'
flux= trial.optimize()
flux.fluxes
# df= flux.to_frame()
# df.loc["EX_glc__D_c"]
# flux.loc["EX_glc__D_c"]

CSm                 2.0
ACONTm              2.0
ICDHxm              2.0
AKGDm               2.0
SUCOASm            28.0
                   ... 
EX_nh4_c            0.0
EX_so3_c            0.0
EX_etoh_c           0.0
EX_glyc_3octa_c     0.0
EX_fru_c            0.0
Name: fluxes, Length: 187, dtype: float64

In [13]:
# trial.reactions.EX_glc__D_c.upper_bound = 57 # μmol substrate /gww.hep/min
#             # https://link.springer.com/article/10.1007/s10439-006-9217-2/tables/2 

# trial.reactions.EX_glc__D_c.lower_bound = -0.585
# trial.reactions.EX_lac__L_c.upper_bound = 3.24 # μmol substrate /gww.hep/min
# # trial.reactions.EX_pyr_c.upper_bound = 3.24

In [14]:
for r in trial.reactions:
    if 'EX_' in r.id:
        print(r.id, r.lower_bound, r.upper_bound)
        r.lower_bound = -1000
        if "glc__D" in r.id:
            r.lower_bound = -0.585 # by convention negative exchange flux = uptake
            r.upper_bound = 57
        elif 'lac__L' in r.id:
            r.upper_bound = 3.24
        elif 'glygn2' in r.id:
            r.upper_bound = 0.304 
        else:
            # r.upper_bound = 1000
            pass
    print(r.id, r.lower_bound, r.upper_bound)

CSm 0.0 1000.0
ACONTm -1000.0 1000.0
ICDHxm 0.0 1000.0
AKGDm 0.0 1000.0
SUCOASm -1000.0 1000.0
SUCD1m -1000.0 1000.0
FUMm -1000.0 1000.0
MDHm -1000.0 1000.0
NADH2_u10mi 0.0 1000.0
FADH2ETC 0.0 1000.0
CYOR_u10mi 0.0 1000.0
CYOOm2i 0.0 1000.0
ATPS4mi 0.0 1000.0
Htmi 0.0 1000.0
HEX1 0.0 1000.0
PGI -1000.0 1000.0
PFK 0.0 1000.0
FBA -1000.0 1000.0
TPI -1000.0 1000.0
GAPD -1000.0 1000.0
PGK -1000.0 1000.0
PGM -1000.0 1000.0
ENO -1000.0 1000.0
PYK 0.0 1000.0
ADK1 -1000.0 1000.0
PDHm 0.0 1000.0
LDH_L -1000.0 1000.0
G6PDH2r -1000.0 1000.0
PGL 0.0 1000.0
GND 0.0 1000.0
RPI -1000.0 1000.0
RPE -1000.0 1000.0
TKT1 -1000.0 1000.0
TKT2 -1000.0 1000.0
TALA -1000.0 1000.0
PCm 0.0 1000.0
PEPCKm 0.0 1000.0
FBP 0.0 1000.0
G6PPer 0.0 1000.0
G6Pter -1000.0 1000.0
GLCter -1000.0 1000.0
MALtm -1000.0 1000.0
MDH -1000.0 1000.0
PEPCK_re 0.0 1000.0
PGMT -1000.0 1000.0
GALUi -1000.0 1000.0
GLGNS1 0.0 1000.0
GLBRAN 0.0 1000.0
GLPASE1 0.0 1000.0
GLDBRAN 0.0 1000.0
GLPASE2 0.0 1000.0
GGNG 0.0 1000.0
ACACT1m 0.0 1000

In [15]:
for r in trial.reactions:
    if 'EX_' in r.id:
        print(r.id, r.lower_bound, r.upper_bound)

EX_h_c -1000 1000.0
EX_pyr_c -1000 1000.0
EX_h2o_c -1000 1000.0
EX_pi_c -1000 1000.0
EX_glc__D_c -0.585 57
EX_lac__L_c -1000 3.24
EX_co2_c -1000 1000.0
EX_o2_c -1000 1000.0
EX_octa_prod_c -1000 1000.0
EX_octa_cons_c -1000 0.0
EX_urea_c -1000 1000.0
EX_gln__L_c -1000 1000.0
EX_acetone_c -1000 1000.0
EX_bhb_c -1000 1000.0
EX_glu__L_c -1000 1000.0
EX_ser__L_c -1000 1000.0
EX_cys__L_c -1000 1000.0
EX_gly_c -1000 1000.0
EX_glygn2_c -1000 0.304
EX_Tyr_ggn_c -1000 1000.0
EX_ala__L_c -1000 1000.0
EX_nh4_c -1000 0.0
EX_so3_c -1000 1000.0
EX_etoh_c -1000 1000.0
EX_glyc_3octa_c -1000 1000.0
EX_fru_c -1000 1000.0


In [22]:
# futile_cycle_1 = trial.problem.Constraint(
#     trial.reactions.PFK.flux_expression - trial.reactions.FBP.flux_expression,
#     lb=-60,
#     ub=60)
# trial.add_cons_vars(futile_cycle_1)


# futile_cycle_2 = trial.problem.Constraint(
#     trial.reactions.HEX1.flux_expression - trial.reactions.G6PPer.flux_expression,
#     lb=-1000,
#     ub=1000)
# trial.add_cons_vars(futile_cycle_2)

# # try doubling the two above because of the 2:1 ratio
# futile_cycle_3 = trial.problem.Constraint(
#     trial.reactions.PYK.flux_expression - trial.reactions.PCm.flux_expression,
#     lb=-20,
#     ub=20)
# trial.add_cons_vars(futile_cycle_3)

In [16]:
dual_objective = trial.problem.Objective(
    -trial.reactions.EX_glygn2_c.flux_expression -
    trial.reactions.EX_glc__D_c.flux_expression,
    direction='min')
trial.objective = dual_objective
solution = trial.optimize(objective_sense='maximise')
# try adding ratios of glycogen to glucose in objective function
# try making the flux reactons of PGI, GLUT2, 
solution

,fluxes,reduced_costs
CSm,0.000,0.0
ACONTm,0.000,0.0
ICDHxm,0.000,0.0
AKGDm,0.000,0.0
SUCOASm,0.000,0.0
...,...,...
EX_nh4_c,0.000,0.0
EX_so3_c,0.000,0.0
EX_etoh_c,0.000,0.0
EX_glyc_3octa_c,0.000,0.0


In [17]:
# #Escher FBA: shows the solutions on the map
initial_flux_data = {
    reaction: flux
    for reaction, flux in solution.fluxes.items()}


#view all the reactions that we need build
escher_builder = escher.Builder(
    # model=trial,
    map_json=os.path.join(
        maps_dir,"core_map_v23.json")
        # ,highlight_missing=True
    , reaction_data=initial_flux_data
    )

# Display map in notebook
escher_builder

Builder(reaction_data={'CSm': 0.0, 'ACONTm': 0.0, 'ICDHxm': 0.0, 'AKGDm': 0.0, 'SUCOASm': 0.0, 'SUCD1m': 0.0, …

In [19]:
#Function to load data from the excel sheet
# def load_data(filepath, sheet_name):
#     """Load Liver data from an excel sheet"""
#     df = pd.read_excel(engine='openpyxl',
#         io=filepath,
#         sheet_name=sheet_name,
#         index_col=0)
#     return df

In [ ]:
#define data sheet as latest data sheet
data_sheet=os.path.join(data_dir,"fluxes.csv")

flux_df = pd.read_csv(
    data_sheet)
flux_df = flux_df.dropna() #drops rows with NaN
flux_df= flux_df.set_index("External Metabolite")
# flux_df.index
flux_df

In [ ]:
for i in flux_df.index:
    # print(i)
    for r in trial.reactions:
        if str(i) in r.id:
    # if i in trial.reactions:
            print(r.id)
            r.lower_bound = flux_df.loc[i,"fasted_lower"]
            r.upper_bound = flux_df.loc[i,"fasted_upper"]
    # print(trial.reactions.__contains__(i))

In [ ]:
for r in trial.reactions:
    if 'EX_' in r.id:
        if r.lower_bound == 0:
            r.lower_bound= -1000
        if r.upper_bound ==0:
            r.upper_bound = 1000
        print(r.id, r.lower_bound, r.upper_bound)

In [ ]:
trial.reactions.EX_glc__D_c.lower_bound = -28.1

In [ ]:
trial.objective = 'ATPS4mi'
flux_solution = trial.optimize()
flux_solution